# NOT USED - DOESNT GO

## Interpolate bed surface


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import time
import datetime as dt
import pandas as pd
import geopandas as gpd
import scipy as sp
from scipy import signal
from tqdm.notebook import tqdm

from shapely.geometry import Point, LineString, Polygon, MultiPoint
import fiona
from shapely.ops import nearest_points

In [18]:
bed_gdf = gpd.read_file('/Volumes/arc_04/FIELD_DATA/K8621920/RES/PROCESSED_LINES/picked_bed.shp')
bed_gdf.DELAY = bed_gdf.DELAY.astype('int').copy()

In [19]:
bed_KIS2 = bed_gdf[bed_gdf.camp=='KIS2'].copy()

In [20]:
bed_KIS2

,CDP,x,y,DELAY,line_name,SEQnum,camp,geometry
0,100,-381564.6,-725999.7,8790,line7,1,KIS2,POINT (-381564.600 -725999.700)
1,101,-381563.4,-725998.3,8790,line7,1,KIS2,POINT (-381563.400 -725998.300)
2,102,-381561.6,-725996.1,8790,line7,1,KIS2,POINT (-381561.600 -725996.100)
3,103,-381559.5,-725993.6,8790,line7,1,KIS2,POINT (-381559.500 -725993.600)
4,104,-381557.4,-725991.1,8790,line7,1,KIS2,POINT (-381557.400 -725991.100)
...,...,...,...,...,...,...,...,...
78350,1617,-380480.6,-728123.0,8280,line5,49,KIS2,POINT (-380480.600 -728123.000)
78351,1618,-380483.0,-728125.5,8280,line5,49,KIS2,POINT (-380483.000 -728125.500)
78352,1619,-380485.3,-728127.9,8280,line5,49,KIS2,POINT (-380485.300 -728127.900)
78353,1620,-380487.6,-728130.1,8280,line5,49,KIS2,POINT (-380487.600 -728130.100)


# Convert all lines to a shapely LineString

In [21]:
linestrings = []
names = []

for line_name in bed_gdf.line_name.unique():
    linestrings.append( LineString(bed_gdf[bed_gdf.line_name==line_name].geometry.tolist() ) )
    names.append(line_name)
#put the lines in a geodataframe
lines_gdf = gpd.GeoDataFrame({'line_name':names},geometry=linestrings)
lines_gdf.set_index('line_name',inplace=True)
lines_gdf.head()

,geometry
line_name,
line7,"LINESTRING (-381564.600 -725999.700, -381563.4..."
line9,"LINESTRING (-380741.900 -721788.100, -380743.5..."
line0,"LINESTRING (-377572.000 -734075.100, -377571.8..."
line10,"LINESTRING (-385757.300 -724442.800, -385756.3..."
line11,"LINESTRING (-384056.400 -720106.700, -384057.5..."


# Interpolate

In [24]:
sp.interpolate.interp2d(bed_KIS2.x, bed_KIS2.y, bed_KIS2.DELAY, kind='linear', copy=True, bounds_error=False, fill_value=None)

/Users/home/whitefar/miniconda3/envs/remote_sensing/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:976: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=376,128 m=46995 fp=5428911201.375602 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [13]:
bed_KIS2.DELAY.iloc[0]

'8790'